## Changelog <br>
<br>
1. All radii are in correct units.<br>
2. More accurate moment of inertia calculated.<br>
3. General code cleanup. <br>
4. Fatal CUDA error magically disappeared.<br>
5. Patch size now scales with radius of amino acid. <br>
6. Setup of sim is now faster by several seconds.<br>
7. Sim box now fits the polymer better. <br>
8. Unused functions in some classes were left in case they could be useful later. <br>


## To Do: <br>
<br>
1. Iterate over LJ sigma for patches (patch_sigma) from kTinput to 10.<br>
2. Iterate over angles (find the maximum and minimum angles where the sim doesn't crash and iterate over those values, 10-20 iterations).<br>
3. Iterate over number of patches (2,4,6,8) and then iterate over the angles and sigmas again.<br>
4. Add bond angles? <br>

## Sequence. Bold = patchy regions: <br>

MR 

AMQ __ALMQIQQ GLQTLATEAP GLIPSFTPGV GVGVLGTAIG PVGPVTPI__ GP

I __GPIVPFT__ PI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT

SESGPN __QQFI QQMVQALAGA NA__ PQLPNPEV RFQQQLEQLN __ANGFL__ NREAN

LQALIATGGD INAAIE __RLLG SQ__ PSW
<br><br>
AMQ __almqiqq glqtlateap glipsftpgv gvgvlgtaig pvgpvtpi__ GP

I __gpivpft__ PI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT

SESGPN __qqfi qqmvqalaga na__ PQLPNPEV RFQQQLEQLN __angfl__ NREAN

LQALIATGGD INAAIE __rllg sq__ PSW

## Full sequence (https://www.uniprot.org/uniprot/Q9UHD9)

        10         20         30         40         50
MAENGESSGP PRPSRGPAAA QGSAAAPAEP KIIKVTVKTP KEKEEFAVPE 

        60         70         80         90        100
NSSVQQFKEA ISKRFKSQTD QLVLIFAGKI LKDQDTLIQH GIHDGLTVHL 

       110        120        130        140        150
VIKSQNRPQG QSTQPSNAAG TNTTSASTPR SNSTPISTNS NPFGLGSLGG 

       160        170        180        190        200
LAGLSSLGLS STNFSELQSQ MQQQLMASPE MMIQIMENPF VQSMLSNPDL 

       210        220        230        240        250
MRQLIMANPQ MQQLIQRNPE ISHLLNNPDI MRQTLEIARN PAMMQEMMRN 

       260        270        280        290        300
QDLALSNLES IPGGYNALRR MYTDIQEPML NAAQEQFGGN PFASVGSSSS 

       310        320        330        340        350
SGEGTQPSRT ENRDPLPNPW APPPATQSSA TTSTTTSTGS GSGNSSSNAT 

       360        370        380        390        400
GNTVAAANYV ASIFSTPGMQ SLLQQITENP QLIQNMLSAP YMRSMMQSLS 

       410        420        430        440        450
QNPDLAAQMM LNSPLFTANP QLQEQMRPQL PAFLQQMQNP DTLSAMSNPR 

       460        470        480        490        500
AMQALMQIQQ GLQTLATEAP GLIPSFTPGV GVGVLGTAIG PVGPVTPIGP 

       510        520        530        540        550
IGPIVPFTPI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT 

       560        570        580        590        600
SESGPNQQFI QQMVQALAGA NAPQLPNPEV RFQQQLEQLN AMGFLNREAN 

       610        620 
LQALIATGGD INAAIERLLG SQPS  

__Notes__: <br>

All but types W and D have patches associated with them.<br>

- __Hydrophobic:__ A, G, I, L, M, P, V <br> 
- __Aromatic:__ F, W, Y <br> 
- __Basic:__ H, K, R <br>
- __Acidic:__ D, E <br>
- __Polar:__ C, N, Q, S, T <br>

## Sources <br>
1. C=N bond length https://www.ncbi.nlm.nih.gov/books/NBK22364/ (no longer used)
2. more bond lengths and angles https://mcl1.ncifcrf.gov/dauter_pubs/171.pdf
3. more bond lengths (see paper I got)
4. Sequence determinants of protein phase behavior from a coarse-grained model.

## Just make rigid centers lowercase and modify seq_list to control each one individually. Can replace entire sections of the sequence with letters if we go more general. 

In [2]:
###Just so the workspace isn't so cramped. 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
import matplotlib.pyplot as plt
from decimal import *
from inspect import currentframe
import string

# class SimBox(self):
class SimBox():
#   def __init__(self, boxsize,nparticles):
#        self.boxsize=boxsize
#        self.nparticles=nparticles
        
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT

class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+(((boxsize-nparticles))/nparticles)*i,0,0]
            system.particles.get(i).diameter = 1.0
            #system.particles.get(i).type = ('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
            patch_types.append('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types

            
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  
class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")

class MomentOfInertia:
  
    #Principle Moment Calculator
    #added an option to return the inertia tensor for debugging
    def Moment(self, coord, Mass, return_tensor = None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        if(return_tensor==True):
            return Idiag[0], Imatrix
        else:
            return Idiag[0]
#Just for flexability 
def Angle(angle):
    angle = angle*np.pi/180
    return angle
DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()
#Filename (date and time will be appended onto the file name)
filename = "V3"
#Number of constituent particles
nparticles = 177 #will need to be 1
#Total number of patches in sim
#tot_npatch = 79
tot_npatch = 4
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
#Empty list for setting up sig values
Sigma = []
#Empty list for setting up eps values
Epsilon = []
#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
#Harmonic Bond Length
hbl = .380
#Temperature/ kT value
Temperature=298
kTinput=Temperature * 8.3144598/1000.
##----------SETUP----------##
context.initialize("")
amino_masses = dict([['A',89.094],['C',121.150],['D',133.103],['E',147.130],['F',165.192],['G',75.067],
                    ['H',155.157],['I',131.175],['K',146.19],['L',131.175],['M',149.210],['N',132.119],
                    ['P',115.132],['Q',146.146],['R',174.204],['S',105.093],['T',119.120],['V',117.148],
                    ['W',204.229],['Y',181.191],
                    ['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
                    ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
                    ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
                    ['w',204.229],['y',181.191]])

#From source [4]
#Old list of amino acid diameters (was not correct in units, kept just in case it's needed)
# amino_diam = dict([['A',5.04],['C',5.48],['D',5.58],['E',5.92],['F',6.36],['G',4.50],
#                     ['H',6.08],['I',6.18],['K',6.36],['L',6.18],['M',6.18],['N',5.68],
#                     ['P',5.56],['Q',6.02],['R',6.56],['S',5.18],['T',5.62],['V',5.86],
#                     ['W',6.78],['Y',6.46],['a',5.04],['c',5.48],['d',5.58],['e',5.92],['f',6.36],['g',4.50],
#                     ['h',6.08],['i',6.18],['k',6.36],['l',6.18],['m',6.18],['n',5.68],
#                     ['p',5.56],['q',6.02],['r',6.56],['s',5.18],['t',5.62],['v',5.86],
#                     ['w',6.78],['y',6.46]])

#Corrected dict of amino acid diameters
amino_diam = dict([['A',0.504],['C',0.548],['D',0.558],['E',0.592],['F',0.636],['G',0.450],
                    ['H',0.608],['I',0.618],['K',0.636],['L',0.618],['M',0.618],['N',0.568],
                    ['P',0.556],['Q',0.602],['R',0.656],['S',0.518],['T',0.562],['V',0.586],
                    ['W',0.678],['Y',0.646],['a',0.504],['c',0.548],['d',0.558],['e',0.592],['f',0.636],['g',0.450],
                    ['h',0.608],['i',0.618],['k',0.636],['l',0.618],['m',0.618],['n',0.568],
                    ['p',0.556],['q',0.602],['r',0.656],['s',0.518],['t',0.562],['v',0.586],
                    ['w',0.678],['y',0.646]])
#Unit charges of the amino acids
amino_charge = dict([['A',0.0],['C',0.0],['D',-1.0],['E',-1.0],['F',0.0],['G',0.0],
                    ['H',0.5],['I',0.0],['K',1.0],['L',0.0],['M',0.0],['N',0.0],
                    ['P',0.0],['Q',0.0],['R',1.0],['S',0.0],['T',0.0],['V',0.0],
                    ['W',0.0],['Y',0.0],['a',0.0],['c',0.0],['d',-1.0],['e',-1.0],['f',0.0],['g',0.0],
                    ['h',0.5],['i',0.0],['k',1.0],['l',0.0],['m',0.0],['n',0.0],
                    ['p',0.0],['q',0.0],['r',1.0],['s',0.0],['t',0.0],['v',0.0],
                    ['w',0.0],['y',0.0]])
 


#Creates dict for patch charges, just placeholder values for the yukawa
pcharge_dict = []
for i in range(tot_npatch):
    pcharge_dict.append(['{0}'.format(i+1), 0.0])
pcharge_dict = dict(pcharge_dict)

#Appends pcharge_dict to amino_charge
amino_charge.update(pcharge_dict)

##Generates all bond types and bond lengths between pairs of amino acids
seq_string = "ACDEFGHIKLMNPQRSTVWYacdefghiklmnpqrstvwy" #List of all amino acids in sim
BL = [] #Bond Lengths
BT = [] #Bond Types
for i in range(len(seq_string)):
    #for j in range(i,len(seq_string)):
    for j in range(len(seq_string)):
        BL.append(['{0}{1}'.format(seq_string[i],seq_string[j]), amino_diam[seq_string[i]]/2. + amino_diam[seq_string[j]]/2. + hbl])
        BT.append('{0}{1}'.format(seq_string[i],seq_string[j]))
BLDic = dict(BL) #Bond Length Dictionary

#Complete Sequence
#sequence = 'MRAMQalmqiqqglqtlateapglipsftpgvgvgvlgtaigpvgpvtpiGPIgpivpftPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNqqfiqqmvqalaganaPQLPNPEVRFQQQLEQLNangflNREANLQALIATGGDINAAIErllgsqPSW'      
sequence = 'MrAMQALMQIQQGLQTLATEAPGLIPSfTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPfTPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNQQfIQQMVQALAGANAPQLPNPEVrfQQQLEQLNANGfLNrEANLQALIATGGDINAAIErLLGSQPSW'
seq_list = []
rigid_tags = []

#Creates a list of the sequence
for i in range(len(sequence)):
    seq_list.append('%s'%sequence[i])
    
#Finds all the rigid bodies (lowercase letters)
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            rigid_tags.append(i)
            
#Creates a list of all the unique amino acids types in the sequence
def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
    # intilize a null list 
    unique_list = [] 
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list


#Creates a list of all amino acid types in polymer
seq_types = unique(seq_list)

pos = []
for ix in range(nparticles):
    if(ix<nparticles-1):
        if(ix!=0):
            pos.append([pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
        elif(ix==0):
            pos.append([-83 + hbl + amino_diam['%s'%seq_list[ix]]/2. +amino_diam['%s'%seq_list[ix+1]]/2. ,0,0])
            #print("diams = {0} {1} {2}".format(hbl,amino_diam['%s'%seq_list[ix]],amino_diam['%s'%seq_list[ix+1]]))
    else:
        pos.append([ pos[ix-1][0] + hbl + amino_diam['%s'%seq_list[ix]]/2. ,0,0])
pos=np.array(pos)

#Creates snapshot, initializes particle types

snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=170, Ly=4, Lz=4),
        bond_types=BT,particle_types=seq_types)
system = hoomd.init.read_snapshot(snapshot)

#Sets each particle's type equal to it's amino acid
for i in range(nparticles):
    system.particles.get(i).type = ('%s'%seq_list[i])
    system.particles.get(i).diameter = amino_diam['%s'%seq_list[i]]

##Adds patch Patch Types from [1,tot_npatch]
##----------Assign Masses----------##
ix=0
for p in system.particles:
    p.mass = amino_masses[p.type]
    p.charge = amino_charge[p.type]

    
    
patch_types = Poly.PatchTypes(tot_npatch, debug=False) 
types = system.particles.types
DB.DebugMessage("DEBUG: types = ", types, debug=True)

patch_sigma = kTinput
##----------Builds Dictionaries for LJ Parameters---------##
for i in range(len(types)):
    if(i>=(len(types)-tot_npatch)):
        Sigma.append(('%s'%types[i], patch_sigma))
        Epsilon.append(('%s'%types[i], 0.5)) 
    elif(i<len(types)):
        Sigma.append(('%s'%types[i], amino_diam[types[i]]))
        Epsilon.append(('%s'%types[i], 0.1))  
sig = dict(Sigma)
DB.DebugMessage("DEBUG: sig = ", sig, debug = True)
eps = dict(Epsilon)
DB.DebugMessage("DEBUG: eps = ", eps, debug = True)


###----------Make Rigid Bodies----------###
rigid=hoomd.md.constrain.rigid()

###Coordinates for patch groups. Not all values used. All coordinates based on the values of the diameters 
###listed in the amino_diam dictionary.
coord_patch_group_a = [[amino_diam['a']*0.5*np.cos(Angle(60)),0,amino_diam['a']*0.5*np.sin(Angle(60))],
                       [amino_diam['a']*0.5*np.cos(Angle(60)),0,-amino_diam['a']*0.5*np.sin(Angle(60))]]

coord_patch_group_l = [[amino_diam['l']*0.5*np.cos(Angle(60)),0,amino_diam['l']*0.5*np.sin(Angle(60))],
                       [amino_diam['l']*0.5*np.cos(Angle(60)),0,-amino_diam['l']*0.5*np.sin(Angle(60))]]

coord_patch_group_m = [[amino_diam['m']*0.5*np.cos(Angle(60)),0,amino_diam['m']*0.5*np.sin(Angle(60))],
                       [amino_diam['m']*0.5*np.cos(Angle(60)),0,-amino_diam['m']*0.5*np.sin(Angle(60))]]

coord_patch_group_q = [[amino_diam['q']*0.5*np.cos(Angle(60)),0,amino_diam['q']*0.5*np.sin(Angle(60))],
                       [amino_diam['q']*0.5*np.cos(Angle(60)),0,-amino_diam['q']*0.5*np.sin(Angle(60))]]

coord_patch_group_k = [[amino_diam['k']*0.5*np.cos(Angle(60)),0,amino_diam['k']*0.5*np.sin(Angle(60))],
                       [amino_diam['k']*0.5*np.cos(Angle(60)),0,-amino_diam['k']*0.5*np.sin(Angle(60))]]

coord_patch_group_i = [[amino_diam['i']*0.5*np.cos(Angle(60)),0,amino_diam['i']*0.5*np.sin(Angle(60))],
                       [amino_diam['i']*0.5*np.cos(Angle(60)),0,-amino_diam['i']*0.5*np.sin(Angle(60))]]

coord_patch_group_g = [[amino_diam['g']*0.5*np.cos(Angle(60)),0,amino_diam['g']*0.5*np.sin(Angle(60))],
                       [amino_diam['g']*0.5*np.cos(Angle(60)),0,-amino_diam['g']*0.5*np.sin(Angle(60))]]

coord_patch_group_t = [[amino_diam['t']*0.5*np.cos(Angle(60)),0,amino_diam['t']*0.5*np.sin(Angle(60))],
                       [amino_diam['t']*0.5*np.cos(Angle(60)),0,-amino_diam['t']*0.5*np.sin(Angle(60))]]

coord_patch_group_e = [[amino_diam['e']*0.5*np.cos(Angle(60)),0,amino_diam['e']*0.5*np.sin(Angle(60))],
                       [amino_diam['e']*0.5*np.cos(Angle(60)),0,-amino_diam['e']*0.5*np.sin(Angle(60))]]

coord_patch_group_p = [[amino_diam['p']*0.5*np.cos(Angle(60)),0,amino_diam['p']*0.5*np.sin(Angle(60))],
                       [amino_diam['p']*0.5*np.cos(Angle(60)),0,-amino_diam['p']*0.5*np.sin(Angle(60))]]

coord_patch_group_s = [[amino_diam['s']*0.5*np.cos(Angle(60)),0,amino_diam['s']*0.5*np.sin(Angle(60))],
                       [amino_diam['s']*0.5*np.cos(Angle(60)),0,-amino_diam['s']*0.5*np.sin(Angle(60))]]

coord_patch_group_f = [[amino_diam['f']*0.5*np.cos(Angle(60)),0,amino_diam['f']*0.5*np.sin(Angle(60))],
                       [amino_diam['f']*0.5*np.cos(Angle(60)),0,-amino_diam['f']*0.5*np.sin(Angle(60))]]

coord_patch_group_v = [[amino_diam['v']*0.5*np.cos(Angle(60)),0,amino_diam['v']*0.5*np.sin(Angle(60))],
                       [amino_diam['v']*0.5*np.cos(Angle(60)),0,-amino_diam['v']*0.5*np.sin(Angle(60))]]

coord_patch_group_n = [[amino_diam['n']*0.5*np.cos(Angle(60)),0,amino_diam['n']*0.5*np.sin(Angle(60))],
                       [amino_diam['n']*0.5*np.cos(Angle(60)),0,-amino_diam['n']*0.5*np.sin(Angle(60))]]

coord_patch_group_r = [[amino_diam['r']*0.5*np.cos(Angle(60)),0,amino_diam['r']*0.5*np.sin(Angle(60))],
                       [amino_diam['r']*0.5*np.cos(Angle(60)),0,-amino_diam['r']*0.5*np.sin(Angle(60))]]


#scaling factor for patches (20% the size of the amino acid)
pscale = 0.2 
rigid_neigh = []
##Gets accurate coordinates for the moment of inertia
##Finds the rigid center in the sequence, gets the coordinates to the right and left of it
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            if((i+1)>=len(sequence)):
                rigid_neigh.append([[amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,-amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [-1.0*np.absolute(pos[i-1][0]-pos[i][0]),0,0],[0,0,0]])
            elif((i+1)<len(sequence) and (i-1)>=0):
                rigid_neigh.append([[amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,-amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [-1.0*np.absolute(pos[i-1][0]-pos[i][0]),0,0],[np.absolute(pos[i][0]-pos[i+1][0]),0,0]])
            elif((i-1)<0):
                rigid_neigh.append([[amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [amino_diam[sequence[i]]*pscale*0.5*np.cos(Angle(60)),0,-amino_diam[sequence[i]]*pscale*0.5*np.sin(Angle(60))],
                    [0,0,0],[np.absolute(pos[i][0]-pos[i+1][0]),0,0]])


mass_patch_groups = []
#reference rigid_tags for indices
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            if((i+1)>=len(sequence)):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]]])
            elif((i+1)<len(sequence) and (i-1)>=0):
                
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i-1]],amino_masses[sequence[i+1]]])
                
            elif((i-1)<0):
                mass_patch_groups.append([0.5,0.5,amino_masses[sequence[i+1]]])

##Calculates the correct moments of inertia to the rigid bodies
eigen_groups = []
for i in range(len(rigid_tags)):
    eigen_groups.append(MoI.Moment(rigid_neigh[i], mass_patch_groups[i], debug=False))


##Setting rigid parameters. 
# rigid.set_param('a',positions=coord_patch_group_a, types=['1','2'], diameters = [amino_diam['a']*pscale,amino_diam['a']*pscale])
# rigid.set_param('l',positions=coord_patch_group_l, types=['3','4'], diameters = [amino_diam['l']*pscale,amino_diam['l']*pscale])
# rigid.set_param('m',positions=coord_patch_group_m, types=['5','6'], diameters = [amino_diam['m']*pscale,amino_diam['m']*pscale])
# rigid.set_param('q',positions=coord_patch_group_q, types=['7','8'], diameters = [amino_diam['q']*pscale,amino_diam['q']*pscale])
# rigid.set_param('i',positions=coord_patch_group_i, types=['9','10'], diameters = [amino_diam['i']*pscale,amino_diam['i']*pscale])
# rigid.set_param('g',positions=coord_patch_group_g, types=['11','12'], diameters = [amino_diam['g']*pscale,amino_diam['g']*pscale])
# rigid.set_param('t',positions=coord_patch_group_t, types=['13','14'], diameters = [amino_diam['t']*pscale,amino_diam['t']*pscale])
# rigid.set_param('e',positions=coord_patch_group_e, types=['15','16'], diameters = [amino_diam['e']*pscale,amino_diam['e']*pscale])
# rigid.set_param('p',positions=coord_patch_group_p, types=['17','18'], diameters = [amino_diam['p']*pscale,amino_diam['p']*pscale])
# rigid.set_param('s',positions=coord_patch_group_s, types=['19','20'], diameters = [amino_diam['s']*pscale,amino_diam['s']*pscale])
# rigid.set_param('v',positions=coord_patch_group_v, types=['23','24'], diameters = [amino_diam['v']*pscale,amino_diam['v']*pscale])
# rigid.set_param('n',positions=coord_patch_group_n, types=['25','26'], diameters = [amino_diam['n']*pscale,amino_diam['n']*pscale])
#rigid.set_param('k',positions=coord_patch_group_f, types=['3','4'], diameters = [amino_diam['k']*pscale,amino_diam['k']*pscale])
rigid.set_param('f',positions=coord_patch_group_f, types=['1','2'], diameters = [amino_diam['f']*pscale,amino_diam['f']*pscale])
rigid.set_param('r',positions=coord_patch_group_r, types=['3','4'], diameters = [amino_diam['r']*pscale,amino_diam['r']*pscale])
rigid.create_bodies()



#Creates group for rigid centers
center = hoomd.group.rigid_center()
#Creates group for nonrigid bodies
nonrigid = hoomd.group.nonrigid()
#Creates group for all particles
part = hoomd.group.all()
#Creates group for all particles that are not patches
gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid)
#Creates group just for the patches
patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly)

#Assignes mass to the patches
for p in patches:
    p.mass = 0.5
#Assigns moments of inertia to the rigid bodies
for i in range(len(rigid_tags)):
    Z = rigid_tags[i]
    system.particles[Z].moment_inertia = eigen_groups[i]

##----------Make Bonds----------##
bi = 0 #bond index
DebugBonds = []
for bi in range(nparticles-1):
    system.bonds.add('{0}{1}'.format(sequence[bi],sequence[bi+1]),bi,bi+1)
    DebugBonds.append([bi,bi+1])
    bi+=1

##Sets the positions of the particles
u=-1
for p in system.particles:
    u+=1
    if(u>=177):
        break
    p.position = pos[u]
    
###----------Make Harmonic Bonds----------###
harm = md.bond.harmonic()
for i in range(len(BT)):
    harm.bond_coeff.set(BT[i],k=3680,r0=BLDic[BT[i]])




###----------Make Angles----------###
###Work in progress, make dict of angles. Look at particle types to deternine the right angle to use. 
    ###Get types: type[i-3],type[i-2],type[i-1]
    ### 
# h=3
# while(h<=(nparticles-1)):
#     system.angles.add("angleA", h-3, h-2, h-1)
#     h+=1
# harmonic = hoomd.md.angle.harmonic()
# harmonic.angle_coeff.set('angleA', k=50.0, t0=Angle(120))

nl = md.nlist.cell(r_buff = 0.25, check_period = 1)
#nl.tune(warmup=200000, r_min=0.05, r_max=1.0, jumps=20, steps=5000, set_max_check_period=False, quiet=False)
yukawa = md.pair.yukawa(r_cut=3.80, nlist=nl)
lj = md.pair.lj(r_cut=rcut, nlist=nl)


##----------Generates LJ pairs----------##
lj_debug = []
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],
                epsilon=0.5*(eps[types[i]]+eps[types[j]]),
                sigma=0.005*(sig[types[i]]+sig[types[j]]))
        lj_debug.append([types[i],types[j],
                'epsilon={0}'.format((eps[types[i]]+eps[types[j]])),
                'sigma={0}'.format((sig[types[i]]+sig[types[j]]))])
DB.DebugMessage("DEBUG: lj_debug = ", lj_debug, debug = False)
##----------Generates Yukawa pairs----------##
##8.98755e9*1e9*((1.6e-19)/2.)**2*6.02e23/1000./80.
yukawa_debug = []
for i in range(0,len(types)):
    for j in range(i,len(types)):
        yukawa.pair_coeff.set(types[i],types[j],
                epsilon=(8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.),
                kappa=(1.0), r_cut=3.5)
        if((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)!=0):
            yukawa_debug.append([types[i],types[j],
                    'epsilon={0}'.format((8.98755e9*1e9*amino_charge[types[i]]*(1.6e-19)*amino_charge[types[j]]*(1.6e-19)*6.02e23/1000./80.)),
                    'kappa={0}'.format(1.0), 'r_cut={0}'.format(3.5)])
DB.DebugMessage("DEBUG: yukawa_debug = ", yukawa_debug, debug = False)


system.replicate(nx=1,ny=3,nz=3)
hoomd.md.integrate.mode_standard(dt=0.001)
#hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=0.kTinput, seed = SEED)
hoomd.md.integrate.langevin(group=nonrigid, kT=kTinput, seed = SEED)
hoomd.md.integrate.langevin(group=center, kT=kTinput, seed = SEED)
# hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
#                                                                'potential_energy','kinetic_energy','translational_kinetic_energy',
#                                                                'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
#                                                                'pair_yukawa_energy','bond_harmonic_energy','angle_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof',
                                                               'potential_energy','kinetic_energy','translational_kinetic_energy',
                                                               'rotational_kinetic_energy','temperature','pressure','pair_lj_energy',
                                                               'pair_yukawa_energy','bond_harmonic_energy'],period=1000,header_prefix='#' , overwrite=True)
hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute','property','momentum','topology'])
hoomd.run(1e6)
